<a href="https://colab.research.google.com/github/rootloginson/X-Ray-Image-Covid19-Detection-Project/blob/master/Final/Data%20Analysis%20CoronaHack%20and%20Making%20Custom%20CSV/DataAnalysis_CoronaHack_and_Make_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
## It'll look better and intuitive on Google Colab Environment (◕‿◕✿)

---

In [ ]:
# This part probably will be enough for utilizing Drive in Colab
# but examine links above if you encounter with problems.
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!nvidia-smi

Mon Jan 18 22:23:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# *Import Libraries and path

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import models
import os 

In [ ]:
# Working Directory path
wd = '/content/drive/MyDrive/Colab Notebooks'

# Creating DataFrame for CustomDataset

### Reading .csv, 
- *Class*(*Labels*) column added (nf: dataframe name )
- Image name setted as index name
- Image dimension list created (gray, rgb, rgba.)
- label_dict (object), for class(label) explanations


In [ ]:
path1 = wd + '/KaggleCovid/Coronahack_csv/Chest_xray_Corona_Metadata.csv'
path2 = wd + '/KaggleCovid/Coronahack_csv/Chest_xray_Corona_dataset_Summary.csv'

# read csv files from Coronahack_csv file
df1 = pd.read_csv(path1, index_col=0)
df2 = pd.read_csv(path2, index_col=0)

In [ ]:
df1

,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN
...,...,...,...,...,...
5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [ ]:
df2

,Label,Label_1_Virus_category,Label_2_Virus_category,Image_Count
0,Normal,NaN,NaN,1576
1,Pnemonia,Stress-Smoking,ARDS,2
2,Pnemonia,Virus,NaN,1493
3,Pnemonia,Virus,COVID-19,58
4,Pnemonia,Virus,SARS,4
5,Pnemonia,bacteria,NaN,2772
6,Pnemonia,bacteria,Streptococcus,5


In [ ]:
# check if there is a null "Label"
df1['Label'].isnull().sum()

0

In [ ]:
# create new frame (nf) with 'class' column
try:
  del nf
except:
  pass

# create copy of the metadata.csv
# nf = Abbr. for 'new frame'
nf = df1.copy()
# add a Class column for classification
nf['Class'] = 'create'

nf.loc[nf['Label']=='Normal', 'Class'] = 0   # Normal
nf.loc[(df1['Label_1_Virus_category']=='bacteria') & (df1['Label_2_Virus_category'].isnull()), 'Class'] = 1            # Bacteria, some
nf.loc[(df1['Label_1_Virus_category']=='Virus') & (df1['Label_2_Virus_category'].isnull()), 'Class'] = 2               # Virus, some
nf.loc[(df1['Label_1_Virus_category']=='Virus') & (df1['Label_2_Virus_category']=='COVID-19'), 'Class'] = 3            # COVID19
nf.loc[(df1['Label_1_Virus_category']=='Virus') & (df1['Label_2_Virus_category']=='SARS'), 'Class'] = 4                # SARS, Virus
nf.loc[(df1['Label_1_Virus_category']=='bacteria') & (df1['Label_2_Virus_category']=='Streptococcus'), 'Class'] = 5    # Streptococcus, Bacteria
nf.loc[(df1['Label_1_Virus_category']=='Stress-Smoking') & (df1['Label_2_Virus_category']=='ARDS'), 'Class'] = 6       # Stress-Smoking

label_dict = {0:'Normal', 1:'Bacteria', 2:'Virus', 3:'Covid19', 4:'Sars, Virus', 5:'Strept., Bacteria', 6:'Stress Smoking'}

In [ ]:
# compare the data summary 
print(df2.info)
print("")
# with created dataframe with class labels
print(nf['Class'].value_counts())

<bound method DataFrame.info of       Label Label_1_Virus_category Label_2_Virus_category  Image_Count
0    Normal                    NaN                    NaN         1576
1  Pnemonia         Stress-Smoking                   ARDS            2
2  Pnemonia                  Virus                    NaN         1493
3  Pnemonia                  Virus               COVID-19           58
4  Pnemonia                  Virus                   SARS            4
5  Pnemonia               bacteria                    NaN         2772
6  Pnemonia               bacteria          Streptococcus            5>

1    2772
0    1576
2    1493
3      58
5       5
4       4
6       2
Name: Class, dtype: int64


```python
# pick 3 random Covid19 image, axis=0 means use row for random pick.
randCovid=list(nf[nf['Class']==3].sample(3, axis=0).X_ray_image_name)

# 3 random covid sample
randCovid = ['F2DE909F-E19C-4900-92F5-8F435B031AC6.jpeg', '1-s2.0-S0929664620300449-gr2_lrg-c.jpg',  'F63AB6CE-1968-4154-A70F-913AF154F53D.jpeg']

for name in randCovid: 
  _here = wd + '/KaggleCovid/Coronahack-Chest-XRay-Dataset/train/' + name
  _there = wd + '/KaggleCovid/Coronahack-Chest-XRay-Dataset/test/' + name   
  os.rename(_here, _there)
```

### Split train and test dataframe
```python
def split_df_train_test(dataframe):
  returns train_df, test_df
```

In [ ]:
def split_df_train_test(dataframe):
  """
  Arguments: pandas dataframe

  Returns  : train_dataframe, test_dataframe 
           : train_dataframe > columns are: 'X_ray_image_name', 'Class'

  """
  nf = dataframe.copy()
  # split dataframe between,(Dataset_type) train and test
  nf_train = nf[nf['Dataset_type']=='TRAIN'].reset_index(drop=True, inplace=False)
  nf_test = nf[nf['Dataset_type']=='TEST'].reset_index(drop=True, inplace=False)
  # create a dataframe with below columns. 
  train_dataframe = nf_train[['X_ray_image_name', 'Class']]
  test_dataframe = nf_test[['X_ray_image_name', 'Class']]

  #check total numbers of datasets are equal 
  a = len(nf.Dataset_type.index)
  b = len(train_dataframe.Class.index)
  c = len(test_dataframe.Class.index)

  # check if splitting true
  print(nf['Dataset_type'].value_counts())
  print("")
  print("train length", len(train_dataframe.index))
  print("test length", len(test_dataframe.index))

  if a == b+c:
    return train_dataframe, test_dataframe
  else:
    return print("Train/Test split failed. Why ? ")

### Image channel dimension list 
list of (gray, rgb, rgba etc.)


In [ ]:
# create copy of nf dataframe to work with
chdim_df = nf.copy()
train_chdim, test_chdim = split_df_train_test(chdim_df)

TRAIN    5286
TEST      624
Name: Dataset_type, dtype: int64

train length 5286
test length 624


In [ ]:
channel_dimensions_dict = {}

# for train file
for i in range(len(train_chdim.index)):
  # corresponding image name
  img_name = train_chdim.iloc[i]['X_ray_image_name'] 
  # open image
  img = Image.open(wd + '/KaggleCovid/Coronahack-Chest-XRay-Dataset/train'+ '/' + img_name)
  # number of channel an image has
  dim = img.getbands()

  if dim not in channel_dimensions_dict:
    channel_dimensions_dict[dim] = []

  # add image name and channel dim to a list. 
  channel_dimensions_dict[dim].append(img_name)
  # I deleted the set after using dictionary

# for test file
for i in range(len(test_chdim.index)):
  # corresponding image name
  img_name = test_chdim.iloc[i]['X_ray_image_name'] 
  # open image
  img = Image.open(wd + '/KaggleCovid/Coronahack-Chest-XRay-Dataset/test'+ '/' + img_name)
  # number of channel an image has
  dim = img.getbands()

  if dim not in channel_dimensions_dict:
    channel_dimensions_dict[dim] = []

  # add image name and channel dim to a list. 
  channel_dimensions_dict[dim].append(img_name)
  # I deleted the set after using dictionary


# {dim:[img_name, ...], dim:[img_name] ...}
print("There are >>", channel_dimensions_dict.keys() , "<< channel Images\n")

# img quantity in channels
dict_info = [(k, len(v)) for k, v in channel_dimensions_dict.items()]
print("image quantity of channels")
print(dict_info)
print("\ntotal")
print(sum(i[1] for i in dict_info))

There are >> dict_keys([('L',), ('R', 'G', 'B'), ('R', 'G', 'B', 'A')]) << channel Images

image quantity of channels
[(('L',), 5565), (('R', 'G', 'B'), 337), (('R', 'G', 'B', 'A'), 8)]

total
5910


In [ ]:
# create a dict of{img_name:img_dim}
reversed_channel_dict = {v_in:len(k) for (k,v) in channel_dimensions_dict.items() for v_in in v}
# create data set of channel dimensions. row: ImageName, column: Channel_dim
dataset_channel_dim_df = pd.DataFrame.from_dict(reversed_channel_dict, orient='index', columns=['Channel_dim'])

print(dataset_channel_dim_df.value_counts())
print("\n size:",dataset_channel_dim_df.size)

Channel_dim
1              5565
3               337
4                 8
dtype: int64

 size: 5910


In [ ]:
save_path_of_channel_dim = wd+'/KaggleCovid/Updated_Data_Frames/dataset_channel_dim.csv'
dataset_channel_dim_df.to_csv(save_path_of_channel_dim, index_label='X_ray_image_name')

### Adding channel dim

In [ ]:
# sorted channel dim, index is image name
channel_dim_list = pd.read_csv(wd+'/KaggleCovid/Updated_Data_Frames/dataset_channel_dim.csv', index_col=0).sort_index()

In [ ]:
# kaggle_dataset, index=X_ray_image_name, column=Class
nf_temp = nf[['X_ray_image_name', 'Dataset_type', 'Class']].set_index('X_ray_image_name').sort_index()

In [ ]:
channel_dim_list.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5910 entries, 01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg to streptococcus-pneumoniae-pneumonia-temporal-evolution-1-day3.jpg
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Channel_dim  5910 non-null   int64
dtypes: int64(1)
memory usage: 92.3+ KB


In [ ]:
nf_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5910 entries, 01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg to streptococcus-pneumoniae-pneumonia-temporal-evolution-1-day3.jpg
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Dataset_type  5910 non-null   object
 1   Class         5910 non-null   object
dtypes: object(2)
memory usage: 138.5+ KB


---
*Adding Channel dims to X_ray_image_name, Class(Label) dataset*


In [ ]:
nf_temp['Channel_dim'] = channel_dim_list['Channel_dim']

# Save Processed csv. 
save_path_of_processed_dataset = wd+'/KaggleCovid/Updated_Data_Frames/processed_dataset.csv'
nf_temp.to_csv(save_path_of_processed_dataset, index_label='X_ray_image_name')

# *Processed Dataframe (--check point)
for processsed dataframe call ***get_processedDataframe()*** function

for kaggle dataframe ***get_kaggleDataframe()*** function
